In [91]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn 

plt.rcParams['font.family'] ='D2Coding'

In [92]:
events = pd.read_csv('events.csv')
order_items = pd.read_csv('order_items.csv')

events = events[events['event_type'] == 'product']
events['product_id'] = events['uri'].str.extract('(\d+)')

events = events[['id', 'user_id', 'product_id', 'created_at']]
order_items = order_items[['user_id', 'product_id', 'created_at']]

# 'user_id'와 'product_id'를 기준으로 -> 두 df를 각각 그룹화하고 -> 각 그룹의 'created_at'의 최소값 찾기
events_new = events.groupby(['user_id', 'product_id'])['created_at'].min().reset_index()
order_items_new = order_items.groupby(['user_id', 'product_id'])['created_at'].min().reset_index()

In [93]:
events_new['product_id'] = pd.to_numeric(events_new['product_id'])

In [94]:
events_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181747 entries, 0 to 181746
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     181747 non-null  float64
 1   product_id  181747 non-null  int64  
 2   created_at  181747 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 4.2+ MB


In [95]:
order_items_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181747 entries, 0 to 181746
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     181747 non-null  int64 
 1   product_id  181747 non-null  int64 
 2   created_at  181747 non-null  object
dtypes: int64(2), object(1)
memory usage: 4.2+ MB


In [96]:
merged_df = pd.merge(events_new, order_items_new, how='outer', on=['user_id', 'product_id'])
merged_df

,user_id,product_id,created_at_x,created_at_y
0,1.0,2953,2022-07-18 10:18:43+00:00,2022-07-20 10:32:05+00:00
1,1.0,4731,2022-07-18 10:54:11+00:00,2022-07-20 11:05:38+00:00
2,1.0,7656,2022-07-18 11:19:54+00:00,2022-07-19 11:29:28+00:00
3,2.0,25774,2022-02-20 10:25:44+00:00,2022-02-20 10:28:57+00:00
4,3.0,18177,2023-03-10 07:10:33+00:00,2023-03-10 07:14:45+00:00
...,...,...,...,...
181742,99998.0,21923,2023-07-22 05:07:51+00:00,2023-07-22 05:16:01+00:00
181743,99999.0,20828,2023-12-22 11:27:35+00:00,2023-12-23 11:33:17+00:00
181744,99999.0,25959,2023-12-22 08:52:25+00:00,2023-12-23 08:59:09+00:00
181745,100000.0,10017,2020-08-28 23:24:14+00:00,2020-08-28 23:27:01+00:00


In [97]:
merged_df.rename(columns = {'created_at_x':'event_created_at', 'created_at_y':'order_item_created_at'}, inplace=True)
merged_df.head()

fin_df = merged_df
fin_df

,user_id,product_id,event_created_at,order_item_created_at
0,1.0,2953,2022-07-18 10:18:43+00:00,2022-07-20 10:32:05+00:00
1,1.0,4731,2022-07-18 10:54:11+00:00,2022-07-20 11:05:38+00:00
2,1.0,7656,2022-07-18 11:19:54+00:00,2022-07-19 11:29:28+00:00
3,2.0,25774,2022-02-20 10:25:44+00:00,2022-02-20 10:28:57+00:00
4,3.0,18177,2023-03-10 07:10:33+00:00,2023-03-10 07:14:45+00:00
...,...,...,...,...
181742,99998.0,21923,2023-07-22 05:07:51+00:00,2023-07-22 05:16:01+00:00
181743,99999.0,20828,2023-12-22 11:27:35+00:00,2023-12-23 11:33:17+00:00
181744,99999.0,25959,2023-12-22 08:52:25+00:00,2023-12-23 08:59:09+00:00
181745,100000.0,10017,2020-08-28 23:24:14+00:00,2020-08-28 23:27:01+00:00


In [98]:
fin_df['event_created_at'] = pd.to_datetime(fin_df['event_created_at'], format='ISO8601')
fin_df['order_item_created_at'] = pd.to_datetime(fin_df['order_item_created_at'], format='ISO8601')
fin_df

,user_id,product_id,event_created_at,order_item_created_at
0,1.0,2953,2022-07-18 10:18:43+00:00,2022-07-20 10:32:05+00:00
1,1.0,4731,2022-07-18 10:54:11+00:00,2022-07-20 11:05:38+00:00
2,1.0,7656,2022-07-18 11:19:54+00:00,2022-07-19 11:29:28+00:00
3,2.0,25774,2022-02-20 10:25:44+00:00,2022-02-20 10:28:57+00:00
4,3.0,18177,2023-03-10 07:10:33+00:00,2023-03-10 07:14:45+00:00
...,...,...,...,...
181742,99998.0,21923,2023-07-22 05:07:51+00:00,2023-07-22 05:16:01+00:00
181743,99999.0,20828,2023-12-22 11:27:35+00:00,2023-12-23 11:33:17+00:00
181744,99999.0,25959,2023-12-22 08:52:25+00:00,2023-12-23 08:59:09+00:00
181745,100000.0,10017,2020-08-28 23:24:14+00:00,2020-08-28 23:27:01+00:00


In [99]:
fin_df = fin_df[fin_df['id'].notnull()]

KeyError: 'id'

In [ ]:
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181747 entries, 0 to 181746
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   user_id                181747 non-null  float64            
 1   product_id             181747 non-null  int64              
 2   event_created_at       181747 non-null  datetime64[ns, UTC]
 3   order_item_created_at  181747 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(1), int64(1)
memory usage: 5.5 MB


In [ ]:
# def get_boolean(df):
#     for col in ['event_created_at']:        
#         if df['event_created_at'] > df['order_item_created_at']:
#             return 1
#         else:
#             return 0

# fin_df.apply(get_boolean)

-----------------------

### 여기부터 over30m 문제

In [103]:
events2 = pd.read_csv('events.csv')
events2 = events2[['session_id', 'created_at']]
events2.head()

,session_id,created_at
0,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00
1,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00
2,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00
3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00
4,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00


In [104]:
events2.sort_values(by = ['session_id', 'created_at'], ascending = True, inplace=True)

In [105]:
events2

,session_id,created_at
1527429,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:26:00+00:00
1090467,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:36:00+00:00
1298355,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:52:00+00:00
1453849,0000364a-ce41-46f1-89d6-3f8704af77db,2022-08-22 10:46:40+00:00
1226266,0000364a-ce41-46f1-89d6-3f8704af77db,2022-08-22 10:49:08+00:00
...,...,...
402596,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:42:12+00:00
1997674,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:43:59+00:00
889988,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:45:41+00:00
327687,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:46:09+00:00


In [108]:
events2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431963 entries, 1527429 to 769437
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   session_id  object
 1   created_at  object
dtypes: object(2)
memory usage: 55.7+ MB


In [110]:
events2['created_at'] = pd.to_datetime(events2['created_at'], format='ISO8601')
events2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431963 entries, 1527429 to 769437
Data columns (total 2 columns):
 #   Column      Dtype              
---  ------      -----              
 0   session_id  object             
 1   created_at  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(1)
memory usage: 55.7+ MB


In [123]:
events2['date_diff'] = events2.groupby(['session_id'])['created_at'].diff().fillna(0)
events2

,session_id,created_at,date_diff
1527429,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:26:00+00:00,0
1090467,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:36:00+00:00,0 days 00:10:00
1298355,00000763-a855-4ad0-a95c-b160e749b272,2021-10-01 03:52:00+00:00,0 days 00:16:00
1453849,0000364a-ce41-46f1-89d6-3f8704af77db,2022-08-22 10:46:40+00:00,0
1226266,0000364a-ce41-46f1-89d6-3f8704af77db,2022-08-22 10:49:08+00:00,0 days 00:02:28
...,...,...,...
402596,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:42:12+00:00,0 days 00:02:09
1997674,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:43:59+00:00,0 days 00:01:47
889988,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:45:41+00:00,0 days 00:01:42
327687,ffffc96a-b665-41a1-8a9a-1101c46e6620,2022-08-25 00:46:09+00:00,0 days 00:00:28


In [124]:
events2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431963 entries, 1527429 to 769437
Data columns (total 3 columns):
 #   Column      Dtype              
---  ------      -----              
 0   session_id  object             
 1   created_at  datetime64[ns, UTC]
 2   date_diff   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 74.2+ MB


In [130]:
events2['date_diff'] = pd.to_timedelta(events2['date_diff'])
events2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2431963 entries, 1527429 to 769437
Data columns (total 3 columns):
 #   Column      Dtype              
---  ------      -----              
 0   session_id  object             
 1   created_at  datetime64[ns, UTC]
 2   date_diff   timedelta64[ns]    
dtypes: datetime64[ns, UTC](1), object(1), timedelta64[ns](1)
memory usage: 74.2+ MB


In [133]:
# events2['over30'] = []

# if events2['date_diff'] >= pd.Timedelta(minutes=30):
#     events2['over30'].append(True)
# else:
#     events2['over30'].append(False)

ValueError: Length of values (0) does not match length of index (2431963)